In [1]:
import pandas as pd

In [2]:
haplotypes_file = "../../input_files/haplotype_mutations.vcf"
haplotypes = pd.read_csv(haplotypes_file,  sep = "\t")

In [3]:
#Creating our haploptypes dictionary 
#only keep SNVs
haplotypes = haplotypes[haplotypes["VARIANT_TYPE"] == "SNV"]

haplotypes = haplotypes.drop_duplicates(subset = ["STRAIN", "START"], \
                                                keep = "first")[["STRAIN", \
                                                                 "START", "REF", "ALT"]]

In [4]:
NZB_haplotypes = haplotypes[haplotypes["STRAIN"] == "NZB"]

#adding the chromosome field
NZB_haplotypes["CHROM"] = "chrM"
NZB_haplotypes["START_1_INDEX"] = NZB_haplotypes["START"] + 1
NZB_haplotypes["END_1_INDEX"] = NZB_haplotypes["START_1_INDEX"] + 1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [5]:
chrM_bed_entries = NZB_haplotypes[["CHROM", "START_1_INDEX", "END_1_INDEX"]] 

In [6]:
chrM_bed_entries_NUMT_region = chrM_bed_entries[(chrM_bed_entries["START_1_INDEX"] >= 6394) & (chrM_bed_entries["START_1_INDEX"] <= 11042)]

In [7]:
len(chrM_bed_entries_NUMT_region)

24

In [8]:
#now we need to create the bed file entries for chr1
#we create this df to locate where we should see differences between chrM (NZB haplotypes) and chr1 NUMT

distance_from_start_NUMT = chrM_bed_entries_NUMT_region[["START_1_INDEX"]]
distance_from_start_NUMT["DIST_FROM_START"] =  distance_from_start_NUMT["START_1_INDEX"] - 6394


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
#create our df of chr1 NUMT info
#these coords are accorinding to the UCSC genome browser (1-indexed)
coords = range(24611535, 24616185)
chrm1_coords = pd.DataFrame(coords , columns = ["START"])
chrm1_coords["END"] = chrm1_coords["START"] + 1
#note: the NUMT region is on the - strand --> the beginning will be the end
chrm1_coords["DIST_FROM_START"] = 24616185 - chrm1_coords["END"] 
chrm1_coords["CHROM"] = "chr1"

In [10]:
#merge our distance_from_start_NUMT df with our chrm1 df to identify which sites we're interested in 

locating_pos_chr1 = pd.merge(distance_from_start_NUMT, chrm1_coords, how = "left", on = "DIST_FROM_START")

chr1_NUMT_pos = locating_pos_chr1[["CHROM", "START", "END"]]
chr1_NUMT_pos.rename(columns = {"START":"START_1_INDEX", "END":"END_1_INDEX"}, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
chr1_NUMT_pos

,CHROM,START_1_INDEX,END_1_INDEX
0,chr1,24616171,24616172
1,chr1,24616108,24616109
2,chr1,24616003,24616004
3,chr1,24615958,24615959
4,chr1,24615793,24615794
5,chr1,24615167,24615168
6,chr1,24615032,24615033
7,chr1,24614708,24614709
8,chr1,24614139,24614140
9,chr1,24614111,24614112


In [12]:
output = pd.concat([chrM_bed_entries_NUMT_region, chr1_NUMT_pos])

In [13]:
output.to_csv('/Users/isabelserrano/Documents/Science/Analyses/Conplastic_Strains/files_and_analyses/checking_NUMT/files/NZB_anchor_points.bed', header=None, index=None, sep='\t')

In [14]:
distance_from_start_NUMT

,START_1_INDEX,DIST_FROM_START
179,6407,13
180,6470,76
181,6575,181
182,6620,226
183,6785,391
184,7411,1017
185,7546,1152
186,7870,1476
187,8439,2045
188,8467,2073


In [15]:
locating_pos_chr1

,START_1_INDEX,DIST_FROM_START,START,END,CHROM
0,6407,13,24616171,24616172,chr1
1,6470,76,24616108,24616109,chr1
2,6575,181,24616003,24616004,chr1
3,6620,226,24615958,24615959,chr1
4,6785,391,24615793,24615794,chr1
5,7411,1017,24615167,24615168,chr1
6,7546,1152,24615032,24615033,chr1
7,7870,1476,24614708,24614709,chr1
8,8439,2045,24614139,24614140,chr1
9,8467,2073,24614111,24614112,chr1
